<a href="https://colab.research.google.com/github/airesx2/Internship-2025/blob/main/Password_Buster_(cudaq).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cudaq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 MB 6.5 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.11.0-py3-none-any.whl size=6897 sha256=fa47cbb5694f997ef816284ccd5ec8ba418db9772d9fabc95aeff8b6435ce3eb
  Stored in directory: /root/.cache/pip/wheels/ad/0f/19/60352710d764963c7c1cb5d5f1ab78dc76e1b542899b33c35a
Successfully built cudaq


In [1]:
import cudaq

In [2]:
#Password Brute Force with grover's algorithm
#Starting with 2 qubits

#Had to move this segment out b/c cudaq didn't support >>
def target_to_bits(target, n):
    #>> shifts bits of a number to the right
    return [(target >> (n - 1 - i)) & 1 for i in range(n)]


@cudaq.kernel #remember the decorator!
def search(bit_pattern: list[int]):

  #Step 1: Create state 0 qubits
  n = len(bit_pattern)
  qubits = cudaq.qvector(n) #initialize n qubits

  #Step 2: Superposition
  for q in qubits:
    h(q) #apply H gate to all qubits

  #looping for better success probability
  for i in range(2):

    #Step 3: Oracle
    for i in range(n): #apply x gates on qubits that should be 0 in the target state
      if bit_pattern[i] == 0:
        x(qubits[i])

    cz(qubits[0], qubits[1]) #flip target state

    for i in range(n): #apply x gates on qubits that should be 0 in the target state
      if bit_pattern[i] == 0:
        x(qubits[i])

    #Step 4: Diffusion
    #reflects the state through origin
    for q in qubits: #puts qubits in "mirror state"
      h(q) #back to superposition
      x(q) #flip qubit

    #reflection step
    cz(qubits[0], qubits[1]) #state vector reflected around the mean amplitude

    for q in qubits: #undo
      x(q)
      h(q)

  #Step 5: Measure
  for q in qubits:
        mz(q)

#Step 6: Run & Sample

#inputs
n = 2 #number of qubits
target = 3
bit_pattern = target_to_bits(target, n)

#run
result = cudaq.sample(search, bit_pattern)
print(f"Result for target = |{target:02b}⟩:")
print(result)


Result for target = |11⟩:
{ 00:239 01:261 10:238 11:262 }

